
> **This notebook implements d-HyMoLAP (a dimensionally consistent reformulation of the HyMoLAP rainfall–runoff model) over CAMELS-FR catchments with less than 10% missing discharge data and stores the resulting calibrated parameters and performance metrics in `dHyMoLAP_Simulation_Data_CAMELS_FR.csv`.**

**Author:** Lionel Cedric Gohouede

# IMPORT LIBRARIES

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from numba import njit
import math
import xarray as xr
import matplotlib.pyplot as plt
from google.colab import drive
from matplotlib.dates import DateFormatter
from scipy.optimize import minimize

##USE DATA FROM PYTHON

In [ ]:
pip install aqua-fetch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 12.9 MB/s eta 0:00:00


In [ ]:
from aqua_fetch import RainfallRunoff

rr = RainfallRunoff("CAMELS_FR")

/usr/local/lib/python3.12/dist-packages/aqua_fetch/rr/utils.py:127: UserWarning: netCDF4 module is not installed. Please install it to save data in netcdf format
  warnings.warn(msg, UserWarning)


downloading 6 files to /usr/local/lib/python3.12/dist-packages/aqua_fetch/data/CAMELS/CAMELS_FR
downloading ADDITIONAL_LICENSES.zip
0% of 0.07 MB downloaded
100% of 0.07 MB downloaded
downloading CAMELS_FR_attributes.zip
0% of 9.88 MB downloaded
100% of 9.88 MB downloaded
downloading CAMELS_FR_geography.zip
0% of 1.45 MB downloaded
100% of 1.45 MB downloaded
downloading CAMELS_FR_time_series.zip
0% of 361.39 MB downloaded
20% of 361.39 MB downloaded
40% of 361.39 MB downloaded
60% of 361.39 MB downloaded
80% of 361.39 MB downloaded
100% of 361.39 MB downloaded
downloading README.md
0% of 0.01 MB downloaded
100% of 0.01 MB downloaded
downloading CAMELS-FR_description.ods
0% of 0.05 MB downloaded
100% of 0.05 MB downloaded
unzipping files in /usr/local/lib/python3.12/dist-packages/aqua_fetch/data/CAMELS/CAMELS_FR
unzipping CAMELS_FR_time_series.zip to CAMELS_FR_time_series
unzipping CAMELS_FR_geography.zip to CAMELS_FR_geography
unzipping CAMELS_FR_attributes.zip to CAMELS_FR_attributes


In [ ]:
meta, ds = rr.fetch()

Read 654 stations for 22 dyn features in 45.41 seconds with 2 cpus.


In [ ]:
print(ds.head())

<xarray.Dataset> Size: 131kB
Dimensions:           (time: 5, dynamic_features: 5)
Coordinates:
  * time              (time) datetime64[ns] 40B 1970-01-01 ... 1970-01-05
  * dynamic_features  (dynamic_features) object 40B 'q_cms_obs' ... 'tsd_val_m'
Data variables: (12/654)
    A105003001        (time, dynamic_features) float64 200B 1.65e+03 ... 10.0
    A107020001        (time, dynamic_features) float64 200B nan nan ... nan nan
    A112020001        (time, dynamic_features) float64 200B 1.04e+03 ... 10.0
    A116003002        (time, dynamic_features) float64 200B nan nan ... nan nan
    A140202001        (time, dynamic_features) float64 200B nan nan ... nan nan
    A202030001        (time, dynamic_features) float64 200B nan nan ... nan nan
    ...                ...
    Y661401001        (time, dynamic_features) float64 200B 860.0 0.441 ... 10.0
    Y781000101        (time, dynamic_features) float64 200B nan nan ... nan nan
    Y862000101        (time, dynamic_features) float64 200B 1.

In [ ]:
print(ds.dynamic_features.values)

['q_cms_obs' 'q_mm_obs' 'tsd_val_s' 'tsd_val_q' 'tsd_val_m' 'tsd_val_c'
 'tsd_val_i' 'pcp_mm' 'pcp_mm_solfrac' 'airtemp_C_mean' 'pet_mm_ou'
 'pet_mm_pe' 'pet_mm_pm' 'windspeed_mps' 'spechum_gkg' 'lwdownrad_wm2'
 'solrad_wm2' 'tsd_swi_gr' 'tsd_swi_isba' 'tsd_swe_isba' 'airtemp_C_min'
 'airtemp_C_max']


In [ ]:
# Full period
print(ds["time"].values[0], ds["time"].values[-1])

1970-01-01T00:00:00.000000000 2021-12-31T00:00:00.000000000


###General

**Functions**

In [ ]:
# ============================================
# dHyMoLAP Model (Highly Optimized)
# ============================================
@njit
def dHyMoLAP_Model(params, Q0, q):
    mu, lambda_, Qs, qs = params
    N = len(q)

    # Initialize state arrays
    k = np.zeros(N)
    x = np.zeros(N)
    r = np.zeros(N)

    # 1. Safety check for qs to prevent division by zero
    if qs > 0.0:
        for t in range(N):
            r[t] = q[t] / qs

    # 2. Initial state
    if Qs > 0.0:
        k[0] = Q0 / Qs
    else:
        k[0] = 0.0

    # 3. Pre-compute constants outside the loop!
    # (Prevents thousands of redundant division operations)
    if lambda_ == 0.0:
        return np.zeros(N) # Safety fallback

    mu_over_lambda = mu / lambda_
    one_minus_mu_over_lambda = 1.0 - mu_over_lambda
    one_over_lambda = 1.0 / lambda_
    power_term = 2.0 * mu - 1.0

    # 4. Main high-speed loop
    for t in range(1, N):

        # Handle missing precipitation data efficiently
        if math.isnan(r[t]):
            k[t] = k[t-1]
            x[t] = x[t-1]
            continue

        # Update routing variable x
        if r[t] > 0.0:
            x[t] = x[t-1] + mu_over_lambda * r[t]
        else:
            x[t] = one_minus_mu_over_lambda * x[t-1]

        # Safety check for exponents: protect against 0^(negative number)
        k_prev = k[t-1]
        if k_prev <= 0.0:
            decay = 0.0
        else:
            decay = mu_over_lambda * (k_prev ** power_term)

        # Update relative discharge k
        k[t] = max(0.0, k_prev - decay + one_over_lambda * x[t] * r[t])

    # Return actual discharge (k * Qs)
    return k * Qs

# ============================================
# Metrics NSE + RMSE (Fast pure NumPy)
# ============================================
def NSE(obs, sim):
    # 1. Create a boolean mask for valid data (drops NaNs at C-speed)
    mask = ~np.isnan(obs) & ~np.isnan(sim)
    valid_obs = obs[mask]
    valid_sim = sim[mask]

    # 2. Safety check: empty arrays or zero variance
    if valid_obs.size == 0 or np.var(valid_obs) == 0.0:
        return np.nan

    # 3. Fast vectorized math
    numerator = np.sum((valid_sim - valid_obs) ** 2)
    denominator = np.sum((valid_obs - np.mean(valid_obs)) ** 2)
    return 1.0 - (numerator / denominator)

def RMSE(obs, sim):
    mask = ~np.isnan(obs) & ~np.isnan(sim)
    valid_obs = obs[mask]
    valid_sim = sim[mask]

    if valid_obs.size == 0:
        return np.nan

    return np.sqrt(np.mean((valid_sim - valid_obs) ** 2))

# ============================================
# Fonction objectif
# ============================================
def objective(params, Q0, q_train, Q_obs_train):
    Q_sim = dHyMoLAP_Model(params, Q0, q_train)
    nse = NSE(Q_obs_train, Q_sim)

    # Return 1 - NSE. If NSE is NaN (due to failed math), return a massive penalty (1e9)
    # to instantly kick the Nelder-Mead optimizer out of that parameter space.
    return 1.0 - nse if np.isfinite(nse) else 1e9

## **Main Code**

In [ ]:
# Optional but highly recommended: Load the slice into RAM to avoid slow disk I/O in the loop
ds_recent = ds.sel(time=slice("2000-01-01", "2021-12-31")).load()

all_stations = list(ds_recent.keys())

b1_ratio = 0.7
max_missing_ratio = 0.1
results = {}

# Replaced manual counter `i` with Python's built-in `enumerate`
for i, station_id in enumerate(all_stations, 1):
    print(f"\n=== Station {station_id} ===, Number = {i}")

    # Extract arrays directly
    Q_obs_raw = ds_recent[station_id].sel(dynamic_features="q_cms_obs").to_numpy()

    # Note: If 'cms' means Cubic Meters per Second, dividing by 1000 is incorrect.
    # Verify if the raw data is actually in L/s before keeping this division.
    Q_obs = Q_obs_raw / 1000.0

    P   = ds_recent[station_id].sel(dynamic_features="pcp_mm").to_numpy()
    PET = ds_recent[station_id].sel(dynamic_features="pet_mm_pm").to_numpy()

    q = np.maximum(0, P - PET)
    N = len(Q_obs)

    if N == 0 or np.all(np.isnan(Q_obs)):
        print("⚠️ Station ignored (no valid data).")
        continue

    missing_count = np.sum(np.isnan(Q_obs))
    missing_ratio = missing_count / N

    if missing_ratio > max_missing_ratio:
        print(f"⚠️ Too many missing values ({missing_ratio*100:.1f}%)")
        continue

    b1 = int(N * b1_ratio)

    # Safe Q0 initialization: Use the first non-NaN observation
    valid_Q_idx = np.where(~np.isnan(Q_obs))[0]
    Q0 = Q_obs[valid_Q_idx[0]] if len(valid_Q_idx) > 0 else 0.0

    # Slice training data once to save memory allocation in the minimize loop
    q_train = q[:b1]
    Q_obs_train = Q_obs[:b1]

    # ============================================
    # Optimization
    # ============================================
    # Bounds logic: mu >= 0.5 (prevent complex math), others > 0 (prevent division by zero)
    param_bounds = [(0.5, 5.0), (1e-3, 100.0), (1e-3, 1000.0), (1e-3, 1000.0)]

    initial_guesses = [
        [1.1, 20.0, np.nanmean(Q_obs_train), np.nanmean(q_train)]
    ]

    best_res = None
    best_val = float("inf")

    for guess in initial_guesses:
        res = minimize(
            objective,
            guess,
            args=(Q0, q_train, Q_obs_train),
            method="Nelder-Mead",
            bounds=param_bounds,
            options={'maxiter': 2500, 'disp': False}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_res = res

    MU, LAMBDA, Qs_best, qs_best = best_res.x
    NSE_train = 1.0 - best_val

    # ============================================
    # Validation
    # ============================================
    # Ensure params are passed as a NumPy array for Numba compatibility
    Qsim = dHyMoLAP_Model(np.array([MU, LAMBDA, Qs_best, qs_best]), Q0, q)

    NSE_val    = NSE(Q_obs[b1:], Qsim[b1:])
    RMSE_train = RMSE(Q_obs_train, Qsim[:b1])
    RMSE_val   = RMSE(Q_obs[b1:], Qsim[b1:])

    print(f"✅ Train NSE: {NSE_train:.3f}, Val NSE: {NSE_val:.3f}")
    print(f"   Train RMSE: {RMSE_train:.3f}, Val RMSE: {RMSE_val:.3f}")
    print(f"   Params: mu={MU:.3f}, lambda={LAMBDA:.3f}, Qs={Qs_best:.3f}, qs={qs_best:.3f}")

    # ============================================
    # Storage
    # ============================================
    results[station_id] = {
        "params": [MU, LAMBDA, Qs_best, qs_best],
        "NSE_train": NSE_train,
        "NSE_val": NSE_val,
        "RMSE_train": RMSE_train,
        "RMSE_val": RMSE_val,
        "Qsim": Qsim,
        "missing_ratio": missing_ratio,
        "missing_count": missing_count,
    }

print(f"\n✅ Simulation complete for {len(results)} valid basins.")


=== Station A105003001 ===, Number = 1
✅ Train NSE: 0.564, Val NSE: 0.601
   Train RMSE: 2.344, Val RMSE: 1.725
   Params: mu=1.104, lambda=31.188, Qs=0.001, qs=0.027

=== Station A107020001 ===, Number = 2
✅ Train NSE: 0.561, Val NSE: 0.510
   Train RMSE: 0.607, Val RMSE: 0.533
   Params: mu=1.116, lambda=30.914, Qs=0.001, qs=0.053

=== Station A112020001 ===, Number = 3
⚠️ Too many missing values (63.5%)

=== Station A116003002 ===, Number = 4
✅ Train NSE: 0.580, Val NSE: 0.679
   Train RMSE: 6.391, Val RMSE: 4.522
   Params: mu=1.079, lambda=21.885, Qs=0.002, qs=0.026

=== Station A140202001 ===, Number = 5
✅ Train NSE: 0.577, Val NSE: 0.626
   Train RMSE: 0.305, Val RMSE: 0.355
   Params: mu=1.224, lambda=19.097, Qs=0.047, qs=1.309

=== Station A202030001 ===, Number = 6
✅ Train NSE: 0.728, Val NSE: 0.808
   Train RMSE: 0.689, Val RMSE: 0.677
   Params: mu=1.215, lambda=23.938, Qs=0.302, qs=1.761

=== Station A204010101 ===, Number = 7
⚠️ Too many missing values (68.1%)

=== Stati

In [ ]:
rows = []

for station_id, res in results.items():
    MU, LAMBDA, Qs, qs = res["params"]

    rows.append({
        "station_id": station_id,
        "MU": MU,
        "LAMBDA": LAMBDA,
        "Qs": Qs,
        "qs": qs,
        "NSE_train": res["NSE_train"],
        "NSE_val": res["NSE_val"],
        "RMSE_train": res["RMSE_train"],
        "RMSE_val": res["RMSE_val"],
    })

df = pd.DataFrame(rows)

# Save locally
df.to_csv("dHyMoLAP_Simulation_Data_CAMELS_FR.csv", index=False)

# Save to Google Drive
from google.colab import drive
drive.mount("/content/drive")

df.to_csv(
    "/content/drive/MyDrive/dHyMoLAP_Simulation_Data_CAMELS_FR.csv",
    index=False
)

df.head()

Mounted at /content/drive


,station_id,MU,LAMBDA,Qs,qs,NSE_train,NSE_val,RMSE_train,RMSE_val
0,A105003001,1.104150,31.187610,0.001000,0.026886,0.564112,0.600765,2.344155,1.725258
1,A107020001,1.116427,30.913879,0.001000,0.053470,0.560999,0.510362,0.607151,0.532757
2,A116003002,1.078674,21.885119,0.001681,0.026183,0.580264,0.679005,6.390789,4.521935
3,A140202001,1.223658,19.097175,0.047484,1.309052,0.577436,0.625659,0.304689,0.354832
4,A202030001,1.214969,23.938059,0.301538,1.760536,0.727864,0.807687,0.688843,0.676642


In [ ]:
nse_values = [res['NSE_val'] for res in results.values()]

if nse_values:
    nse_mean = np.mean(nse_values)
    nse_min = np.min(nse_values)
    nse_max = np.max(nse_values)
    nse_5th = np.percentile(nse_values, 5)
    nse_95th = np.percentile(nse_values, 95)

    print(f"NSE Validation -> Mean: {nse_mean:.3f}, Min: {nse_min:.3f}, Max: {nse_max:.3f}, 5th percentile: {nse_5th:.3f}, 95th percentile: {nse_95th:.3f}")
else:
    print("No stations processed (all contain missing values).")


NSE Validation -> Mean: 0.649, Min: -0.270, Max: 0.900, 5th percentile: 0.411, 95th percentile: 0.835


In [ ]:
rmse_values = [res['RMSE_val'] for res in results.values()]

if rmse_values:
    rmse_50th = np.percentile(rmse_values, 50)
    rmse_5th = np.percentile(rmse_values, 5)
    rmse_95th = np.percentile(rmse_values, 95)

    print(f"RMSE Validation -> Median: {rmse_50th:.3f}, 5th percentile: {rmse_5th:.3f}, 95th percentile: {rmse_95th:.3f}")
else:
    print("No stations processed (all contain missing values).")


RMSE Validation -> Median: 1.702, 5th percentile: 0.255, 95th percentile: 17.046
